In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
%matplotlib inline

In [2]:
dataL = pd.read_csv("https://raw.githubusercontent.com/ComputoCienciasUniandes/MetodosComputacionalesDatos/master/homework/2017-10/planetas/lowp_finalresults.csv")
dataH = pd.read_csv('https://raw.githubusercontent.com/ComputoCienciasUniandes/MetodosComputacionalesDatos/master/homework/2017-10/planetas/highp_finalresults.csv')
dataL.head()

,ident,it,t,a(i),emegas(i),emepla(i)/emet,rplanet(i)/radtie,emestar,rc,qest,sigmag_0,emed,gama,apert,fpert,constmigI,emetal,taugas
0,0,199992,20000001.0,0.026231,0.001185,1.710156,1.8388847169925493,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
1,0,199992,20000001.0,2.170653,0.000000,1.646707,1.8167126017756039,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
2,0,199992,20000001.0,4.251030,0.000000,1.059631,1.5688831343393115,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
3,0,199992,20000001.0,5.468018,0.000000,0.839156,1.4520602505243447,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0
4,0,199992,20000001.0,8.935596,0.000000,0.367578,1.1032029110256696,4.368174e+32,59.88237,1.110726,51.287674,0.13,1.0,0.1,1.0,0.1,0.07707,9268935.0


In [3]:
X_L = dataL.drop(['emegas(i)','emepla(i)/emet'],inplace=False,axis=1).groupby(['ident']).mean()
X_H = dataH.drop(['emegas(i)','emepla(i)/emet'],inplace=False,axis=1).groupby(['ident']).mean()

In [4]:
#varibales independientes: 
#numero de planetas
Planetas_H = dataH.groupby(['ident']).size().get_values()
Planetas_L = dataL.groupby(['ident']).size().get_values()
#mas total de planetas
masa_H = dataH.groupby(['ident'])['emepla(i)/emet'].sum().get_values()
masa_L = dataL.groupby(['ident'])['emepla(i)/emet'].sum().get_values()
#eficiencia de masa: 
Em_H = masa_H/dataH.groupby(['ident'])['emed'].mean().get_values()
Em_L = masa_L/dataL.groupby(['ident'])['emed'].mean().get_values()

In [5]:
scaler = preprocessing.StandardScaler().fit(X_H)
scaler.transform(X_H)

array([[ 0.31019953,  0.        ,  1.49602166, ..., -1.26950241,
         0.40339121,  2.04608931],
       [ 0.24635915,  0.        , -0.71532753, ..., -0.6347512 ,
        -0.38453134, -0.19516564],
       [ 0.31031026,  0.        ,  2.83438615, ..., -1.26950241,
         2.43371264,  2.09349159],
       ..., 
       [-1.09545156,  0.        , -0.79698783, ..., -0.6347512 ,
        -0.13235122, -0.161002  ],
       [ 0.30399821,  0.        ,  0.08178531, ..., -1.26950241,
        -0.74662793,  1.92465158],
       [ 0.30366599,  0.        , -0.03435897, ..., -1.26950241,
         1.93943245,  1.43797598]])

In [6]:
scaler = preprocessing.StandardScaler().fit(X_L)
scaler.transform(X_L)

array([[ 0.3167378 ,  0.        ,  2.09856586, ..., -1.04720195,
         0.39214288,  2.04063229],
       [ 0.26248232,  0.        , -0.77194317, ..., -1.04720195,
        -0.39885956, -0.19632769],
       [ 0.31690171,  0.        ,  2.49744347, ..., -1.04720195,
         2.43040056,  2.08794374],
       ..., 
       [-1.13597595,  0.        , -0.83510441, ..., -1.04720195,
        -0.14569369, -0.16222951],
       [ 0.3088153 ,  0.        ,  0.34193732, ..., -1.04720195,
        -0.76237153,  1.91942728],
       [ 0.31116473,  0.        ,  0.12942822, ..., -1.04720195,
         1.93418829,  1.43368431]])

In [9]:
XPH_train, XPH_test, YPH_train, YPH_test = train_test_split(X_H, Planetas_H)
XmH_train, XmH_test, YmH_train, YmH_test = train_test_split(X_H, masa_H)
XEH_train, XEH_test, YEH_train, YEH_test = train_test_split(X_H, Em_H)

In [99]:
XPL_train, XPL_test, YPL_train, YPL_test = train_test_split(X_L, Planetas_L)
XmL_train, XmL_test, YmL_train, YmL_test = train_test_split(X_L, masa_L)
XEL_train, XEL_test, YEL_train, YEL_test = train_test_split(X_L, Em_L)

In [20]:
def gauss(C,gamma,X_train,X_test,Y_train,Y_test):
    kernel_svr = svm.SVR(C=C, kernel='rbf', gamma=gamma)
    kernel_svr.fit(X_train,Y_train)
    rr=kernel_svr.score(X_test,Y_test)
    return rr

In [103]:
def solve_gauss(X_train,X_test,Y_train,Y_test):
    C=np.linspace(1000,10000,4)
    gamma=np.linspace(0.00001,1000,4)
    n=len(C)*len(gamma)
    RR=[]
    for i in range(len(C)):
        for j in range(len(gamma)):
            rr= gauss(C[i],gamma[j],X_train,X_test,Y_train,Y_test)
            RR.append(rr)
    return RR 

In [26]:
def lineal(C,X_train, X_test, Y_train, Y_test):
    linear_svr = svm.SVR(C=C, kernel='linear')
    linear_svr.fit(X_train,Y_train)
    rr=linear_svr.score(X_test,Y_test)
    return rr

In [30]:
def solve_linear(X_train, X_test, Y_train, Y_test):
    C=np.linspace(1,1000,16)
    RR=[]
    for i in range(len(C)):
        rr= lineal(C[i],X_train, X_test, Y_train, Y_test)
        RR.append(rr)
    return RR 

In [78]:
from sklearn import tree
def trees(X_train, X_test, Y_train, Y_test,m):
    trees=tree.DecisionTreeRegressor(criterion=m)
    trees.fit(X_train, Y_train)
    print (trees.score(X_test,Y_test))

In [95]:
from sklearn import neural_network
def neural_network(X_train, X_test, Y_train, Y_test):
    neural=neural_network.MLPRegressor()
    neural.fit(X_train,Y_train)
    print (neural.score(X_test,Y_test))
neural_network(XPH_train, XPH_test, YPH_train, YPH_test)

AttributeError: 'function' object has no attribute 'MLPRegressor'

In [ ]:
trees(XPH_train, XPH_test, YPH_train, YPH_test,'mse')
trees(XPH_train, XPH_test, YPH_train, YPH_test,'mae')
solve_gauss(XPH_train, XPH_test, YPH_train, YPH_test)

0.405338171934


In [ ]:
trees(XmH_train, XmH_test, YmH_train, YmH_test,'mse')
trees(XmH_train, XmH_test, YmH_train, YmH_test,'mae')
solve_gauss(XmH_train, XmH_test, YmH_train, YmH_test)

In [ ]:
trees(XEH_train, XEH_test, YEH_train, YEH_test,'mse')
trees(XEH_train, XEH_test, YEH_train, YEH_test,'mae')
solve_gauss(XEH_train, XEH_test, YEH_train, YEH_test)

In [ ]:
trees(XPL_train, XPL_test, YPL_train, YPL_test,'mse')
trees(XPL_train, XPL_test, YPL_train, YPL_test,'mae')
solve_gauss(XPL_train, XPL_test, YPL_train, YPL_test)

In [ ]:
trees(XmL_train, XmL_test, YmL_train, YmL_test,'mse')
trees(XmL_train, XmL_test, YmL_train, YmL_test,'mae')
solve_gauss(XmL_train, XmL_test, YmL_train, YmL_test)

In [ ]:
trees(XEL_train, XEL_test, YEL_train, YEL_test,'mse')
trees(XEL_train, XEL_test, YEL_train, YEL_test,'mae')
solve_gauss(XEL_train, XEL_test, YEL_train, YEL_test)